# Homework: Named entity recognition

Для заданной тестовой выборки построить модель для обнаружения и классификации именованных сущностей (named entities). На базе корпуса CoNLL 2002.  

Чем больше baseline'ов вы превзойдете, тем выше ваша оценка
Метрика качества f1 (f1_macro) (чем выше, тем лучше)
 
baseline 1: 0.0604      random labels  
baseline 2: 0.3966      PoS features + logistic regression  
baseline 3: 0.7559      word2vec cbow embedding + baseline 2 + svm    

Пока мы рассмотрели только линейные модели - поэтому в примерах есть только они. Желательно при решении домашнего задания пользоваться линейными моделями. Таким образом, основные цели задания - feature engineering, hyperparam tuning & model selection.

! Your results must be reproducible. Если ваша модель - стохастическая, то вы явно должны задавать все seed и random_state в параметрах моделей  
! Вы должны использовать df_test только для измерения качества конечной обученной модели. 

bonus, think about:  
1. how can you exploit that words belong to some sentence?
2. why we selected f1 score with macro averaging as our classification quality measure? What other metrics are suitable   

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

In [2]:
df = pd.read_csv('./data/ner_short.csv', index_col=0)
df.head()

,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [3]:
# number of sentences
df.sentence_idx.max()

1500.0

In [4]:
# class distribution
df.tag.value_counts(normalize=True )

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [5]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [6]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [7]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
0,1.0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48
1,1.0,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48
2,1.0,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48
3,1.0,9,through,32,marched,33,18,9,of,demonstrators,have,O,48
4,1.0,16,London,9,through,32,33,18,demonstrators,have,marched,O,48


In [8]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [9]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict
#from glove import Corpus, Glove

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    

class GloveWrapper(TransformerMixin):
    def __init__(self, window=5, learning_rate=0.05, size=100, epochs=100, random_state=SEED, verbose=False):
        self.window_ = window
        self.learning_rate_ = learning_rate
        self.size_ = size
        self.epochs_ = epochs
        self.glove = None
        self.random_state = random_state
        self.verbose_ = verbose
    
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        corpus = Corpus()
        corpus.fit(sentences_list, window=self.window_)
        self.glove = Glove(no_components=self.size_, 
                           learning_rate=self.learning_rate_, 
                           random_state=self.random_state)
        self.glove.fit(corpus.matrix, 
                  epochs=self.epochs_, 
                  no_threads=4, verbose=self.verbose_)
        self.glove.add_dictionary(corpus.dictionary)
        return self
    
    def has(self, word):
        return word in self.glove.dictionary

    def transform(self, X):
        """
        X: list of words
        """
        if self.glove is None:
            raise Exception('model not fitted')
            
        return np.array([self.glove.word_vectors[self.glove.dictionary[w]] if w in self.glove.dictionary else np.zeros(self.size_) 
                         for w in X])

In [10]:
%%time
# we have to fit embedding models on whole dataset as they depend on word order
from gensim.models.word2vec import Word2Vec


# notice, that our dataset has window=2
sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

w2v_cbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

CPU times: user 21.1 s, sys: 454 ms, total: 21.5 s
Wall time: 7.39 s


In [13]:
def counts(column,Y):
    counts = dict()
    for target in np.unique(Y):
        counts[target] = dict()
        for elem in np.unique(column):
            counts[target][elem] = (max(0,(((column == elem) & (Y == target)).sum())))/((column == elem).sum())
    return counts

In [14]:
def to_counts(name):
    counts_dict = counts(df_train[name],y_train)
    counts_val_train = []
    for i in range(df_train.shape[0]):
        counts_val_train.append(counts_dict[y_train[i]][df_train[name].values[i]])
    counts_val_test = []
    for i in range(df_test.shape[0]):
        counts_val_test.append(counts_dict[y_test[i]][df_test[name].values[i]])
    return counts_val_train, counts_val_test

In [15]:
counts_train = []
counts_test = []
for col in ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']:
    counts_val_train, counts_val_test = to_counts(col)
    counts_train.append(counts_val_train)
    counts_test.append(counts_val_test)
X_counts_train = pd.DataFrame(np.array(counts_train).T,columns=['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos'])
X_counts_test = pd.DataFrame(np.array(counts_test).T,columns=['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos'])

In [16]:
X_counts_train

,pos,next-pos,next-next-pos,prev-pos,prev-prev-pos
0,1.000000,0.864933,0.771429,0.656173,0.769064
1,1.000000,0.886589,0.870662,0.926883,0.908356
2,0.997922,0.964962,0.915347,0.926883,0.846100
3,0.997922,0.879809,0.896825,0.752735,0.846100
4,0.093364,0.089918,0.011631,0.015037,0.020552
5,0.997922,0.879809,0.782200,0.752735,0.828609
6,0.996438,0.928810,0.873957,0.987038,0.828609
7,0.985690,0.964962,0.747801,0.987038,0.890899
8,0.985690,0.633261,0.782200,0.960905,0.909440
9,0.993836,0.781049,0.726111,0.987038,0.830663


In [17]:
import scipy.sparse as sp

embedding = w2v_cbow
X_train = sp.hstack([
    embedding.transform(df_train.word),
    embedding.transform(df_train['next-word']),
    embedding.transform(df_train['next-next-word']),
    embedding.transform(df_train['prev-word']),
    embedding.transform(df_train['prev-prev-word']),
    X_counts_train])
X_test = sp.hstack([
    embedding.transform(df_test.word),
    embedding.transform(df_test['next-word']),
    embedding.transform(df_test['next-next-word']),
    embedding.transform(df_test['prev-word']),
    embedding.transform(df_test['prev-prev-word']),
    X_counts_test])

In [18]:
X_train

<50155x1505 sparse matrix of type '<class 'numpy.float64'>'
	with 57925775 stored elements in COOrdinate format>

In [25]:
%%time
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(penalty='l2', cv=5, multi_class='ovr', random_state=SEED, n_jobs=8,solver='lbfgs')
model.fit(X_train, y_train)


CPU times: user 4min 35s, sys: 7.41 s, total: 4min 43s
Wall time: 46min 20s


In [26]:
print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

train 0.967905376066
test 0.818954521891
